In [1]:
def eval_dimensions_sem(task, pred):
    lengths = [float(x) for x in task['output'].split('\n')[0].split()]
    mse = np.mean(abs(np.array(lengths) - np.array(pred[0]))/np.array(lengths))
    angles = [float(x) for x in task['output'].split('\n')[1].split()]
    mae = np.mean(abs(np.array(angles) - np.array(pred[1]))/np.array(angles))

In [6]:
def eval_dimensions(task, pred):
    # task = dimensions, pred is a list
    if "lengths of the lattice vectors" in task['input']:
        lengths = [float(x) for x in task['output'].split(',')]
        mse = np.mean(abs(np.array(lengths) - np.array(pred))/np.array(lengths))
        return mse
    else:
        angles = [float(x) for x in task['output'].split(',')]
        mae = np.mean(abs(np.array(angles) - np.array(pred))/np.array(angles))
        return mae

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('17000_dimensions_synt_llama2.csv')

In [17]:
# for lis in df.pred_cs:
#     try:
#         lengths = [float(x) for x in lis]
#     except:
#         print(lis)

In [11]:
lengths

[3.3, 5.6, 10.8]

In [1]:
import sys
print(sys.executable)

/home/cse/btech/cs1200448/.conda/envs/llamat3_infer/bin/python


In [2]:
import json
import numpy as np
import pandas as pd
import string
import re
import collections
from sklearn.metrics import f1_score
import Levenshtein
import os
from tqdm import tqdm


def most_similar_answer(a,answer_set):
    a = a.strip().replace(' ', '')
    if(a in answer_set):
        return a
    dis = [Levenshtein.distance(a,x) for x in answer_set]
    idx = np.argmin(dis)
    return answer_set[idx]

def eval_atom_cnt(task, pred):
    # task = atom count
    return pred==task['output']

def eval_dimensions(task, pred):
    # task = dimensions, pred is a list
    if "lengths of the lattice vectors" in task['input']:
        lengths = [float(x) for x in task['output'].split(',')]
        mse = np.mean(abs(np.array(lengths) - np.array(pred))/np.array(lengths))
        return mse
    else:
        angles = [float(x) for x in task['output'].split(',')]
        mae = np.mean(abs(np.array(angles) - np.array(pred))/np.array(angles))
        return mae

def eval_atom_name(task, pred):
    # task = atom name
    return pred==task['output']

def eval_spacegroup(task, pred):
    # task = space group
    return pred==task['output']

def eval_cell_volume(task, pred):
    # task = cell_volume
    return abs(float(task['output'])-pred)/float(task['output'])

def eval_formula(task, pred):
    # task = formula
    return task['output']==pred

def eval_replace(task, pred):
    # task = replace
    answer = most_similar_answer(pred, ["Yes", "No"])
    return task['output']==answer

def eval_dimensions_sem(task, pred):
    lengths = [float(x) for x in task['output'].split('\n')[0].split()]
    mse = np.mean(abs(np.array(lengths) - np.array(pred[0]))/np.array(lengths))
    angles = [float(x) for x in task['output'].split('\n')[1].split()]
    mae = np.mean(abs(np.array(angles) - np.array(pred[1]))/np.array(angles))

    return mse, mae

def eval_infill_task(task, pred):
    return pred==task['output']

def eval_gen_format(pred):
    def trim_list(l):
        return [x.strip().replace('\n', '') for x in l if x]
    l = trim_list(pred.split('\n'))
    l1 = trim_list(l[0].split())
    l2 = trim_list(l[1].split())
    if len(l1)!=3 or len(l2)!=3:
        return 0
    matrix = l[2:]
    if len(matrix)%2:
        return 0
    for i in range(0, len(matrix), 2):
        l1 = trim_list(matrix[i].split())
        l2 = trim_list(matrix[i+1].split())
        if len(l1)!=1 or len(l2)!=3:
            return 0
        for x in l2:
            try:
                y = float(x)
            except:
                return 0
    return 1



In [3]:
valfile = []
with open("/scratch/cse/btech/cs1200448/MatLlama/ift_cif_large/val.jsonl", 'r') as f:
    valfile = [json.loads(line) for line in f.readlines()]

out_dict = dict()
tasks = ["atom count", "dimensions_synt", "atom name", "replace", "space group", "cell_volume", "formula", "dimensions_sem", "vol_calc"]
# "infill", "formula_compute", "conditional_generation", "element_generation"
for task in tasks:
    out_dict[task] = []

idxs = []
for _, sample in enumerate(valfile):
    task = sample['task']
    system = sample['system']
    idxs.append(_)
    with open(f'/home/cse/btech/cs1200448/MatLlama/cif_infer_outputs_cs/{_}.txt', 'r') as f:
        output = f.read()

    if task in tasks:
        out_dict[task].append([output, sample])
    elif task=="dimensions":
        if "predict" not in system and "forecast" not in system:
            out_dict["dimensions_synt"].append([output, sample])
        else:
            out_dict["dimensions_sem"].append([output, sample])  

In [4]:
_

27182

In [6]:
scores = dict()
for task in ['atom name']:
    print(task, end=':')
    scores[task] = 0
    if task=="dimensions_sem":
        scores[task] = [0, 0]
    true_cnt = 0 if ("dimensions" in task or "volume" in task) else len(out_dict[task])
    for output, sample in out_dict[task]:
        if task=="atom count":
            output = output.strip().replace('\n', '').replace(' ', '')
            if output.isdigit():
                scores[task] += eval_atom_cnt(sample, int(output))
        if task=="dimensions_synt":
            l = output.split(',')
            l = [x for x in l if x!='']
            l = list(map(lambda x: x.strip().replace('\n', '').replace(' ', ''), l))
            if len(l)==3:
                try:
                    scores[task] += eval_dimensions(sample, [float(l[0]), float(l[1]),float(l[2])])
                    true_cnt += 1
                except:
                    continue
        if task=="atom name":
            output = output.strip().replace('\n', '').replace(' ', '')
            scores[task] += eval_atom_name(sample, output)
        if task=="replace":
            scores[task] += eval_replace(sample, output)
        if task=="space group":
            output = output.strip().replace('\n', '').replace(' ', '')
            scores[task] += eval_spacegroup(sample, output)
        if task=="cell_volume":
            output = output.strip().replace('\n', '').replace(' ', '')
            try:
                answer = float(output)
                scores[task] += eval_cell_volume(sample, answer)
                true_cnt += 1
            except:
                continue
        if task=="formula":
            output = output.strip().replace('\n', '').replace(' ', '')
            scores[task] += eval_formula(sample, output)
        if task=="infill":
            output = output.strip().replace('\n', '').replace(' ', '')
            scores[task] += eval_infill_task(sample, output)
        if task=="dimensions_sem":
            l1 = output.split('\n')
            l1 = [x for x in l1 if x!='']
            if len(l1)!=2:
                continue
            l21 = l1[0].split()
            l22 = l1[1].split()
            l21 = [x.strip() for x in l21 if x!='']
            l22 = [x.strip() for x in l22 if x!='']
            if len(l21)==3 and len(l22)==3:
                mse, mae = eval_dimensions_sem(sample, [[float(l21[0]), float(l21[1]),float(l21[2])], [float(l22[0]), float(l22[1]), float(l22[2])]])
                true_cnt += 1
                scores[task][0] += mse
                scores[task][1] += mae
        if task=="vol_calc":
            output = output.strip().replace('\n', '').replace(' ', '')
            try:
                answer = float(output)
                scores[task] += eval_cell_volume(sample, answer)
                true_cnt += 1
            except:
                continue 
        if task=="formula_compute":
            output = output.strip().replace('\n', '').replace(' ', '')
            scores[task] += eval_formula(sample, output)
        if "generation" in task:
            scores[task] += eval_gen_format(output)
        
    if "dimensions_sem" not in task:
        scores[task] /= true_cnt
    else:
        scores[task][0] /= true_cnt
        scores[task][1] /= true_cnt

    print(scores[task], true_cnt, len(out_dict[task]))

atom name:0.2534672405547585 2091 2091


<h2>dimensions_sem</h2>

In [73]:
task = 'dimensions_sem'
df1k = pd.read_csv(f'1000_{task}_llama3.csv')
df15k = pd.read_csv(f'15000_{task}_llama3.csv')

In [74]:
import re

def extract_numbers(s):
    # Use regex to find floating-point numbers
    result = re.findall(r'\d+\.\d+|\d+', s)
    # Join the result list into a single string
    return float(''.join(result))


In [75]:
true_cnt = 0
scores = dict()

scores[task] = [0, 0]
for i in range(len(df1k)):

    sample = {'output':df1k.actual.iloc[i], 'input':df15k.input.iloc[i]}
    output = df1k.pred_cs.iloc[i]
    l1 = output.split('\n')
    l1 = [x for x in l1 if x!='']
    if len(l1)!=2:
        continue
    l21 = l1[0].split()
    l22 = l1[1].split()
    l21 = [x.strip() for x in l21 if x!='']
    l22 = [x.strip() for x in l22 if x!='']
    # print(l22)
    if len(l21)==3 and len(l22)==3:
        mse, mae = eval_dimensions_sem(sample, [[float(l21[0]), float(l21[1]),float(l21[2])], [float(l22[0]), float(l22[1]), float(extract_numbers(l22[2]))]])
        true_cnt += 1
        scores[task][0] += mse
        scores[task][1] += mae
    else:
        print(i)

In [76]:
scores

{'dimensions_sem': [88.03136787216347, 120.99035484565152]}

In [77]:
scores[task][0]/len(df1k), scores[task][1]/len(df1k)

(0.042100128107203955, 0.05786243655937424)

<h2>vol_calc</h2>

In [78]:
task = 'vol_calc'
df1k = pd.read_csv(f'1000_{task}_llama3.csv')
df15k = pd.read_csv(f'15000_{task}_llama3.csv')

In [79]:
import re

def extract_numbers(s):
    # Use regex to find floating-point numbers
    result = re.findall(r'\d+\.\d+|\d+', s)
    # Join the result list into a single string
    return float(''.join(result))


In [83]:
true_cnt = 0
scores = dict()

scores[task] = 0
for i in range(len(df1k)):

    sample = {'output':df1k.actual.iloc[i], 'input':df15k.input.iloc[i]}
    output = df1k.pred_cs.iloc[i]
    # output = output.split('Title')[0]
    # output = extract_numbers(output)
    try:
        answer = float(output)
        scores[task] += eval_cell_volume(sample, answer)
        true_cnt += 1
    except:
        continue

In [84]:
scores

{'vol_calc': 27.402442642897732}

In [85]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [86]:
for func in [r2_score, mean_absolute_error, mean_squared_error]:
    print(func(df1k.actual, df1k.pred_cs))

for func in [r2_score, mean_absolute_error, mean_squared_error]:
    print(func(df15k.actual, df15k.pred_cs))

0.9970438392499362
5.71646775539933
212.70589630630772
0.9995139077623317
3.0016779670636065
34.97600226866867


In [77]:
scores[task][0]/len(df1k), scores[task][1]/len(df1k)

(0.042100128107203955, 0.05786243655937424)

<h2>cell_volume</h2>

In [42]:
task = 'cell_volume'
df1k = pd.read_csv(f'15000_{task}_llama3.csv')
df15k = pd.read_csv(f'15000_{task}_llama3.csv')

In [43]:
true_cnt = 0
scores = dict()

scores[task] = 0
for i in range(len(df1k)):
    # if task=="cell_volume":
    output = str(df1k.pred_cs.iloc[i])
    output = output.strip().replace('\n', '').replace(' ', '')
    sample = {'output':df1k.actual.iloc[i], 'input':df15k.input.iloc[i]}
    try:
        answer = float(output)
        scores[task] += eval_cell_volume(sample, answer)
        true_cnt += 1
    except Exception as e:
        print(i,e)
        continue

In [44]:
scores

{'cell_volume': 0.0}

<h2>dimensions_synt</h2>

In [20]:
true_cnt = 0
scores = dict()

scores[task] = 0
for i in range(len(df1k)):
    output = df1k.pred_cs.iloc[i]
    
    l = output.split(',')
    l = [x for x in l if x!='']
    l = list(map(lambda x: x.strip().replace('\n', '').replace(' ', ''), l))
    sample = {'output':df1k.actual.iloc[i], 'input':df15k.input.iloc[i]}
    
    if len(l)==3:
        try:
            scores[task] += eval_dimensions(sample, [float(l[0]), float(l[1]),float(l[2])])
            true_cnt += 1
        except Exception as e:
            print(i, e)
            # continue

In [23]:
scores[task]/2091

0.002874286991686685